# Tamizh Data Cleaning

This Code is extract useful information form Tamizh data-set from (Wikimedia). 

Informations Planned to extract:
  
  1. Nouns
  2. multiple word - similar meaning
  3. Categories
  
  
 Data leaning part
  1. Extract only tamil sentence

Import relevant libraries

In [8]:
import pandas as pd
import re
import regex

Load the data

In [9]:
data = pd.read_csv("../data/tawiki_pages_articles.csv")

Define variable to save all the extracted information

1. Wikipedia article clean texts
2. Nouns (here nouns are considered as value bounded in box)
3. Curl Values
4. Heading values for articles
5. Tamil - English combined
6. Categories

In [31]:
#This will be considered as global variables #hope we have enought RAM for this slliness

boxed_values = []
curlBoxed_values = []
heading_values = []
boxed_and_curls = []
categories = []
all_nouns = []

Define regular expressoin syntax

And extract information functions



In [11]:
re_boxed_values = "\[[^\]^\[]*\]"
re_curlBoxed_values = "\{[^\{]*\}"
re_heading_values = "\=\=([^\=]*)\=\="
re_boxed_and_curls = "\[[^\]^\[]*\]\] \([^\(^\)]*\)"
re_categories = "\[\[பகுப்பு:([^\]]*)\]"
  

Cleaning is required with multiple string needs to be eliminated with regex

Here I used "regex" function to extract tamil chars from the string (Note : not re)

In [45]:
#set of regex rules to extract patterns

re_braces = "\([^\)]*\)"
re_curl_braces ="\{\{[^\}\{]*\}\}"
re_single_curl = "\{[^\}\{]*\}"
# re_curl_braces ="\{[^\}]*\}"
# re_double_curl_braces = "\{[^\{]*\{[^\}]*\}\}[^\}]*\}\}"
re_category = "\[\[பகுப்பு:[^\]]*\]"
re_header = "(\=\=[^\=]*\=\=)"
re_img = "\[\[படிமம்:.*\]\]"
re_ref1 = "<ref.*?/>"
re_ref2 = "<ref.*?</ref>"

def extract_nouns(text):
  # Extract nouns only Tamil
  text_r = text
  boxed = re.findall( r"\[\[(.*?)\]\]",text)
  nouns = []
  for box in boxed:
      if ':' not in box:
          if "|" in box:
              nouns.append(box.split("|")[0])
              text_r = text_r.replace(box,box.split("|")[0])
          else:
              nouns.append(box)
          
  # filter only tamil
  nouns = [noun for noun in nouns if regex.match(r"[\p{Tamil} .]",noun)]
  
  
  # return noun in replaced text
  return nouns , text_r

def parse_data(text,title):
  global boxed_values, curlBoxed_values, heading_values, boxed_and_curls, categories, all_nouns
  boxed_values.append({'title':title,'box':list(set(re.findall(re_boxed_values,text,re.M)))})
  curlBoxed_values.append({'title':title,'curl':list(set(re.findall(re_curlBoxed_values,text,re.M)))})
  heading_values.append({'title':title,'heading':list(set(re.findall(re_heading_values,text,re.M)))})
  boxed_and_curls.append({'title':title,'boxcurl':list(set(re.findall(re_boxed_and_curls,text,re.M)))})
  categories.append({'title':title,'categories':list(set(re.findall(re_categories,text,re.M)))})
  nouns , text_r = extract_nouns(text)
  all_nouns += nouns
  all_nouns = list(set(all_nouns))
  return text_r

def clean_data(text):
  text = re.sub(re_braces," ",text)
  text = re.sub(re_ref2," ",text)
  text = re.sub(re_ref1," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_single_curl," ",text)
  text = re.sub(re_category," ",text)
  text = re.sub(re_header," ",text)
  text ="".join(regex.findall("[\p{Tamil} ]",text))
  text = re.sub("\s\s+" , " ", text)
  return text

Even some entire sections in the article is not needed for the language modelling purpose. 

We can eliminate those from the existance :-p 

In [46]:
section_rm= ['மேற்கோள்கள்','இவற்றையும் பார்க்கவும்','வெளி இணைப்புகள்','நூற்பட்டியல்','துணை நூல்கள்']

def remove_sections(text):
  text = re.sub(re_img,"",text)
  article = ""
  for para in text.split("\n\n"):
    if not any([section in para for section in section_rm]):
      article += para 
      
  return article

In [47]:
#remove_sections(data.text[8])

So Here we go !!!!!!!!!!!!

We have all the funtions for 

1. Extract the relevant information from the article
2. Clean the data
3. Extracting the pure -(information) from the article 


In [48]:
#combine all these into a single funciton

def extract_tawiki(text,title):
  if type(text)==str:
    text = remove_sections(text)
    text = parse_data(text,title)
    clean_text = clean_data(text)
    return clean_text
  else:
    return ""
  

Apply the same logic to all data in the dataframe 



In [49]:
#text data
tawiki_data= []


#Other useful information
boxed_values = []
curlBoxed_values = []
heading_values = []
boxed_and_curls = []
categories = []
all_nouns = []




In [50]:
extract_tawiki(data.text.iloc[300],data.title.iloc[0])

' செவ்வாய் சூரியக் குடும்பம் உள்ள ஒரு கோள் ஆகும் இது சூரியன் நான்காவது கோளாக உள்ளதுஇக்குடும்பத்தில் மிகச்சிறிய கோளான புதன் அடுத்ததாக இரண்டாவது சிறிய கோளாக செவ்வாய் இருக்கிறது மேனாட்டினர் இக்கோளுக்கு போர்க்கடவுளின் பெயரைச் சூட்டியுள்ளனர் இதன் மேற்பரப்பில் காணப்படும் இரும்பு ஆக்சைடு இக்கோளைச் செந்நிறமாகக் காட்டுகிறது இதனாலேயே இதற்குச் செவ்வாய் என்ற பெயர் ஏற்பட்டது ஒரு புவிசார் கோள் இதன் மேற்பரப்பு சந்திரனில் உள்ளதுபோல் கிண்ணக் குழிகளையும் புவியில் உள்ளது போன்ற எரிமலைகள் பள்ளத்தாக்குகள் பாலைவனம் பனிமூடிய துருவப் பகுதிகளையும் கொண்டது செவ்வாயின் சுழற்சிக்காலம் பருவ மாற்றங்களும் புவிக்கு உள்ளதைப் போன்றவையே சூரிய மண்டலம் மிக உயரமான ஒலிம்பசு மலையும் மிகப்பெரிய செங்குத்துப் பள்ளத்தாக்குகளுள் ஒன்றான மரினர் பள்ளத்தாக்கும் செவ்வாயிலேயே உள்ளன ஆம் ஆண்டில் செவ்வாய்க்கு அண்மையாக மரினர் வெற்றிகரமாகப் பறந்து செல்லும்வரை செவ்வாய்க் கோளின் மேற்பரப்பில் நீர்ம நீர் இருக்கும் என நம்பினர் கோளின் துருவப் பகுதிகளுக்கு அருகில் குறித்த கால அடிப்படையில் மாற்றம் அடைகின்ற கறுப்பு வெள்ளை அடையாளங்களே இவ்வாறான நம்பிக்க

In [18]:
a = data.text.iloc[300]

In [31]:
nouns

['ஹபிள் விண்வெளித் தொலைநோக்கி',
 'ஞாயிறு (விண்மீன்)',
 'நிலநடுக் கோடு',
 'செவ்வாயின் இயற்கைத் துணைக்கோள்கள்',
 'கெல்வின்',
 'பிரித்தானிய ஆங்கிலம்',
 'அமெரிக்க ஆங்கிலம்',
 'பாசுக்கல் (அலகு)',
 'கார்பனீராக்சைடு',
 'ஆர்கான்',
 'நைட்ரசன்',
 'ஆக்சிசன்',
 'கார்பனோராக்சைடு',
 'சூரியக் குடும்பம்',
 'கோள்',
 'சூரியன்',
 'புதன் (கோள்)',
 'இரும்பு ஆக்சைடு',
 'புவிசார் கோள்',
 'கிண்ணக் குழி',
 'எரிமலை',
 'பள்ளத்தாக்கு',
 'பாலைவனம்',
 'சுழற்சிக்காலம்',
 'சூரிய மண்டலம்',
 'ஒலிம்பசு மலை',
 'மரினர் பள்ளத்தாக்கு',
 'மரினர் 4',
 'கடல்',
 'கண்டம்',
 'ஒளியியல் மாயத்தோற்றம்',
 'போபொசு (துணைக்கோள்)',
 'போபொசு (துணைக்கோள்)',
 'பூமி',
 'சிலிக்கன்',
 'ஆக்சிசன்',
 'உலோகங்கள்',
 'பாறை',
 'தனிமம்',
 'கனிமம்',
 'சிலிக்காக் கண்ணாடி',
 'பைரொட்சீன்',
 'காரத்தன்மை',
 'மக்னீசியம்',
 'சோடியம்',
 'பொட்டாசியம்',
 'குளோரைடு',
 'நீர்மம்',
 'நிலத்தடி நீத்தேக்கம்',
 'எரிமலைக் குழம்பு',
 'நீளுருண்டை']

In [32]:
print(data.text.iloc[300])

{{Infobox planet
| name = செவ்வாய்
| symbol = [[File:Mars symbol (bold).svg|24px|♂]]
| bgcolour= #a0ffa0
| image = [[File:Mars 23 aug 2003 hubble.jpg|275px|The planet Mars]]
| caption = Mars imaged by the<br />[[ஹபிள் விண்வெளித் தொலைநோக்கி]] in 2003
| orbit_ref = <ref name="VSOP87" />
| epoch = [[J2000]]
| aphelion = {{val|fmt=commas|1.6660|ul=AU}}<br />{{val|fmt=commas|249.2|u=Gm}}
| perihelion = {{val|fmt=commas|1.3814|u=AU}}<br />{{val|fmt=commas|206.7|u=Gm}}
| semimajor = {{val|fmt=commas|1.523679|u=AU}}<br />{{val|fmt=commas|227.9392|u=Gm}}
| eccentricity = 0.0934
| period = {{val|1.8808|u=[[julian year (astronomy)|Julian years]]}}<br />{{val|686.971|u=days}}<br />{{val|668.5991}}&nbsp;[[timekeeping on Mars|sols]]
| synodic_period = 779.96&nbsp;days<br />2.135&nbsp;Julian years
| avg_speed = {{val|24.077|u=km/s}}
| mean_anomaly = {{val|19.373|u=°}}
| inclination = 1.850° to [[ecliptic]]<br />5.65° to [[ஞாயிறு (விண்மீன்)]]'s [[நிலநடுக் கோடு]]<br />1.67° to [[invariable plane]]<ref 

In [9]:
for i in range(data.shape[0]):
  tawiki_data.append(extract_tawiki(data.text.iloc[i],data.title.iloc[i]))

NameError: name 'extract_tawiki' is not defined

In [10]:
#data to be saved
tawiki_data_df = pd.DataFrame(tawiki_data,columns=['Content'])
boxed_values_df = pd.DataFrame(boxed_values)
#curl_boxed_values = pd.DataFrame(curlBoxed_values)
boxed_and_curls_df = pd.DataFrame(boxed_and_curls)
categories_df = pd.DataFrame(categories)

In [11]:
categories_df.head()

,title,categories
0,முதற் பக்கம்,[]
1,விக்கிப்பீடியா:கலந்துரையாடல்,[விக்கிப்பீடியா ஆலமரத்தடி]
2,விக்கிப்பீடியா:புதுப் பயனர் பக்கம்,[விக்கிப்பீடியா உதவி]
3,விக்கிப்பீடியா:விக்கிப்பீடியர்கள்,[விக்கிப்பீடியர்கள்]
4,கட்டிடக்கலை,[கட்டிடக்கலை| ]


Save all the results to disk.



In [12]:
tawiki_data_df.to_csv("../data/tawiki_data_df.csv",index=False)
boxed_values_df.to_csv("../data/boxed_value_df.csv",index =False)
boxed_and_curls_df.to_csv("../data/boxednCurls_df.csv",index=False)
categories_df.to_csv("../data/categories_df.csv",index=False)